In [1]:
from owslib.fes import *
from owslib.etree import etree
from owslib.wfs import WebFeatureService
import re
import geopandas as gpd
import requests
from owslib.wms import WebMapService

In [2]:
# getting layers of data for streets
url = "https://wms2.geopoz.poznan.pl/geoserver/topografia/ows?SERVICE=WMS&REQUEST=GetCapabilities"
r = requests.get(url)
wms = WebMapService(url, version='1.3.0')

In [ ]:
layers = list(wms.contents)
for layer in layers[40:]:
    # print only this with (poligon) in title or without any brackets
    if "poligon" in wms[layer].title or "(" not in wms[layer].title:
        print(layer, wms[layer].title)

In [4]:
list_of_layers = ["tereny_komunikacyjne_e_sql", "tereny_wodne_e_sql", "tereny_wodne_sql", "zwj_poly_sql", "zwr_poly_sql"]
layers_titles = ["streets", "water_areas1", "water_areas2", "staying_water", "running_water"]
dict_of_layers = dict(zip(list_of_layers, layers_titles))

In [5]:
wfs11 = WebFeatureService(url='https://wms2.geopoz.poznan.pl/geoserver/topografia//wfs', version='1.1.0')

In [6]:
[operation.name for operation in wfs11.operations]

['GetCapabilities', 'DescribeFeatureType', 'GetFeature', 'GetGmlObject']

In [ ]:
# read all layers to gpd but rememmber about adding 'topografia:' before layer name
for layer, title in dict_of_layers.items():
    typename = 'topografia:' + layer
    output_format = 'GML2'
    response = wfs11.getfeature(typename=typename, outputFormat=output_format)
    with open(title + '.gml', 'wb') as file:
        file.write(response.read())
    gdf = gpd.read_file(title + '.gml')

In [8]:
# getting layers of data for streets
url = "https://wms2.geopoz.poznan.pl/geoserver/transport//wfs?service=wfs&version=2.0.0&request=GetCapabilities"
wms = WebMapService(url, version='1.3.0')
layers = list(wms.contents)

In [9]:
 # regular expressions
for layer in layers:
    # print only this with "park" in title, use regex to find "park" in title
    if re.search("park", wms[layer].title, re.IGNORECASE):
        print(layer, wms[layer].title)

parkomaty_sql Transport - Parkomaty
spp_sektory Transport - Strefa Platnego Parkowania (sektory dla mieszkancow)
strefy_platnego_parkowania_sql Transport - Strefa Platnego Parkowania
v_parkingi_sql Transport - Parkingi dla rowerów


#### chyba nie ma parkingów :(